In [1]:
########
## Env Setup / Preamble
########
%reload_ext autoreload
%autoreload complete 

%reload_ext Cython

%display latex

################

import cython
from sage.all import *

################

import tempfile
import mmap
from multiprocessing import shared_memory

################

import numpy as np
import scipy.constants

np.random.seed( int(scipy.constants.golden_ratio * 1.e6) )

from numpy import vectorize
from numpy import r_, c_, s_

################

####    boiler-plate class for named constants
from sage.symbolic.constants import Constant
class ConstantFromExpression(Constant):
    def __init__(self, name, v):
        conversions = dict(maxima=repr(maxima.coerce(v)))
        Constant.__init__(self, name, conversions=conversions)
        self._v = v
    def __float__(self): return float(self._v)
    def __complex__(self): return complex(self._v)
    def _mpfr_(self, R): return R(self._v)
    def _real_double_(self, R): return R(self._v)

def NamedExpression(name, expr):
    return ConstantFromExpression(name,expr).expression()
####

import sage.symbolic.units as units     # <: TODO

𝛑   = NamedExpression('𝛑', scipy.constants.pi)         # pi
𝛆_0 = NamedExpression('𝛆_0', scipy.constants.epsilon_0) # electric permittivity
𝛍_0 = NamedExpression('𝛍_0', scipy.constants.mu_0)     # magnetic permeability of vacuum

𝛋 = 1. / (4 * 𝛑 * 𝛆_0)      # Coulomb's constant in [ N·m²/C² ]
𝙦_e = NamedExpression('𝙦_e', -scipy.constants.elementary_charge)

𝒉_planck = NamedExpression('𝒉', scipy.constants.Planck)
ℏ = NamedExpression('ℏ', scipy.constants.hbar)

𝛂 = NamedExpression('𝛂', scipy.constants.fine_structure)

𝒎_e = NamedExpression('𝒎_e', scipy.constants.electron_mass)
𝒎_p = NamedExpression('𝒎_p', scipy.constants.proton_mass)

amu = NamedExpression(
    'amu', 
    scipy.constants.physical_constants['atomic mass constant'][0]
)

################

#Parallelism().set(nproc=8)
print(f"Python: {sys.version}")
version()


Python: 3.12.5 (main, Feb 25 2025, 12:37:41) [Clang 16.0.0 (clang-1600.0.26.6)]


'SageMath version 10.6, Release Date: 2025-03-31'

# [7]

do a rotation on a 2D metric (like the circle/polar problem) and show same answer

# [14]

Figure out the metric for a 2D curved earth with longitude/latitude coordinates.

In [2]:
var("R")
S2 = manifolds.Sphere(2, radius=R)
R3 = S2.ambient()

spherical.<𝛉, 𝛗> = S2.spherical_coordinates()
spherical


Chart (A, (theta, phi))

In [3]:
# South Pole, 𝛉 = 𝛑
spherical.coord_range()


theta: (0, pi); phi: [-pi, pi] (periodic)

In [4]:
cartesian.<x,y,z> = R3.chart()

cartesian


Chart (E^3, (x, y, z))

In [5]:
𝚽 = S2.embedding()
𝚽.display()


iota: S^2_R → E^3
on A: (theta, phi) ↦ (x, y, z) = (R*cos(phi)*sin(theta), R*sin(phi)*sin(theta), R*cos(theta))

In [6]:
g = S2.induced_metric()
g[:]


[             R^2                0]
[               0 R^2*sin(theta)^2]

In [7]:
g.display()


g = R^2 dtheta⊗dtheta + R^2*sin(theta)^2 dphi⊗dphi

In [8]:
# South Pole, West - Negative Sign 
# North Pole, East - Positive Sign

𝛉_l = lambda l: (𝛑/2) + ((-𝛑 / 180.) * l) 
𝛗_𝛌 = lambda 𝛌: (𝛑 / 180.) * 𝛌

#latlong.<𝛌,l> = S2.chart(r'𝛌:[-180,180] l:[-90,90]')
#latlong.transition_map(
    #spherical,
    #[
        #(𝛑/2) + ((-𝛑 / 180.) * l),  # latitude to 𝛉
        #(𝛑 / 180.) * 𝛌,             # longitude to 𝛗
    #]
#)


In [9]:
# TODO :> need to show metric `g` in terms of <𝛌,l>


# [15]

Find EoM for and solve it for Denver to London

---

Recall that the metric is given by:

$$ ds^2 = R^2\ \partial \theta^2 + R^2\ \sin^2\theta \partial \phi^2 $$

For now, we can assume $R = 1$, which simplifies to 

$$ ds^2 = (1)\ \partial \theta^2 + (\sin^2\theta)\ \partial \phi^2 $$


Now recall that the Geodesic Equation is given by 

$$ 
    \frac{\partial^2}{\partial \tau^2} x^\mu +  
        \Gamma^{\mu}_{\alpha \beta} 
        \frac{\partial x^\alpha}{\partial \tau}
        \frac{\partial x^\beta}{\partial \tau}
        = 0
    \\ \Leftrightarrow \\
    \ddot{\gamma}^\mu + \Gamma^{\mu}_{\alpha \beta}
    \dot{\gamma}^\alpha \dot{\gamma}^\beta
    = 0 
$$

where the "proper-time" $\tau = s$, our implicit path-length variable of the metric. 

For our given system, the Christoffel symbols $\Gamma^\mu_{\alpha\beta}$ are:

In [38]:
g.christoffel_symbols_display(
    coordinate_labels=False,
    only_nonredundant=False
)


Gam^1_22 = -cos(theta)*sin(theta) 
Gam^2_12 = cos(theta)/sin(theta) 
Gam^2_21 = cos(theta)/sin(theta)

Or equivalently, 

$$\begin{aligned} 
\Gamma^\theta_{\phi\phi} &= -\sin\left(\theta\right)\cos\left(\theta\right) \\
\Gamma^\phi_{\theta\phi} = \Gamma^\phi_{\phi\theta} &= \cot\left(\theta\right) \\
\Rightarrow \Gamma^\phi_{\alpha\beta} &= 2 \cot\theta
\end{aligned}$$

We may now simplify the geodesic equation, indexed by $\mu \in \{\theta, \phi\}$, to receive the 
equations of motion in the coordinate system: 

$$
\begin{align}
    \ddot{\theta}\ &-\ \sin\left(\theta\right)\cos\left(\theta\right) &\left(\dot\phi\right)^2 &= 0 \\
    \ddot{\phi}\ &+\ 2 \cot\left(\theta\right) &\dot\theta\dot\phi &= 0
\end{align}
$$

We will begin with Eqn. (2) above;

$$\begin{split}
    \ddot{\phi}\ +\ 2 \cot\left(\theta\right)\ \dot\theta\dot\phi = 0 & \\
    &\Rightarrow^* \sin^2\theta\ \ddot{\phi} + 2 \sin\theta \cos\theta\ \dot\theta\dot\phi = 0 \\
    &\Rightarrow^* \partial_\tau \left( \sin^2\theta\ \dot\phi \right) = 0 \qquad 
    \textrm{By Product and Chain Rules} \\
    &\Rightarrow^* \sin^2\theta\ \dot\phi = c_1 \qquad \textrm{By Integration}\\
    &\Rightarrow^* \dot\phi = c_1 \csc^2\theta \\
\end{split}$$

where $c_1$ is a constant of integration. 

Back-substituting into Eqn. (1):

$$\begin{split}
    \ddot{\theta}\ -\ \sin\left(\theta\right)\cos\left(\theta\right) \left(\dot\phi\right)^2 = 0 & \\
    &\Rightarrow^* \ddot{\theta}\ -\ \sin\left(\theta\right)\cos\left(\theta\right) \left(c_1\ \csc^2\theta\right)^2 = 0 \\
    &\Rightarrow^* \ddot{\theta} - \left(c_1\right)^2 \cot\theta \csc^2\theta = 0 \\
    &\Rightarrow^* \ddot{\theta} = \left(c_1\right)^2 \cot\theta \csc^2\theta \\
    &\Rightarrow^* \dot{\theta} = \left(c_1\right)^{2} \left(c_2 - \cot\theta\right) \qquad \textrm{By Integration} \\
\end{split}$$

where $c_2$ is a constant of integration. 

Since we are not given any information about the initial 'velocities' $c_1, c_2$, 
we must progress further. 

Substituting and simplifying:

$$\begin{split}
\left[\begin{align*}
    \ddot\theta &= \left(c_1\right)^2 \cot\theta \csc^2\theta \\
    \ddot\phi &= -2\cot\theta \left(c_1\right)^2 \left(c_2 - \cot\theta\right)(c_1 \csc^2\theta) \\
\end{align*}\right]
    \quad &\Leftrightarrow \quad 
\left[\begin{align*}
    \ddot\theta &= \left(c_1\right)^2 \cot\theta \csc^2\theta \\
    \ddot\phi &= -2\cot\theta \left(\left(c_1\right)^2 \left(c_2 - \cot\theta\right)\right)(c_1 \csc^2\theta) \\
\end{align*}\right] \\
    \quad &\Leftrightarrow \quad 
\left[\begin{align*}
    \ddot\theta &= \left(c_1\right)^2 \cot\theta \csc^2\theta \\
    \ddot\phi &= -2\ (c_1)^3 \csc^2\theta \cot\theta \left(c_2 - \cot\theta \right) \\
\end{align*}\right]
\end{split}$$

In [10]:
denver = S2.point( 
    (
        𝛉_l(39.73915), 
        𝛗_𝛌(-104.9847),
    ), 
    name="Denver"
)

print(denver)
spherical(denver)


Point Denver on the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3


(0.279226944444444*𝛑, -0.583248333333333*𝛑)

In [11]:
london = S2.point( 
    (
        𝛉_l(51.50853),
        𝛗_𝛌(-0.12574), 
    ), 
    name="London"
)

print(london)
spherical(london)


Point London on the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3


(0.213841500000000*𝛑, -0.000698555555555556*𝛑)

---

Equivalently, we may engage the computer algebra system using the Manifolds 
construction as follows:

In [30]:
var('s')

v = S2.tangent_vector(denver,   # point
                      (         # direction, ```London - Denver```
                          spherical(london) + 
                          (-1 * spherical(denver))
                      ), 
)

[s, s_min, s_max] = var('s smin smax')

geodesic = S2.integrated_geodesic(
    g,
    (s, s_min, s_max),
    v,
    chart=spherical,
    name="geodesic"
)

geo = geodesic.system(verbose=True)
geo


Geodesic geodesic in the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3 equipped with Riemannian metric g on the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3, and integrated over the Real interval (smin, smax) as a solution to the following geodesic equations, written with respect to Chart (A, (theta, phi)):

Initial point: Point Denver on the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3 with coordinates [0.279226944444444*𝛑, -0.583248333333333*𝛑] with respect to Chart (A, (theta, phi))
Initial tangent vector: Tangent vector at Point Denver on the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3 with components [0.213841500000000*𝛑, -0.000698555555555556*𝛑] with respect to Chart (A, (theta, phi))

d(theta)/ds = Dtheta
d(phi)/ds = Dphi
d(Dtheta)/ds = Dphi^2*cos(theta)*sin(theta)
d(Dphi)/ds = -2*Dphi*Dtheta*cos(theta)/sin(theta)



[[Dphi^2*cos(theta)*sin(theta), -2*Dphi*Dtheta*cos(theta)/sin(theta)],
 Tangent vector at Point Denver on the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3,
 Chart (A, (theta, phi))]

That is, we may express the equations of motion in the coordinate system as

$$
\begin{array}{rclc}
    \frac{\partial^2 \theta}{\partial s^2} &=&\ \sin\theta\cos\theta\ &\partial\phi^2 \\ 
    \frac{\partial^2 \phi}{\partial s^2} &=&\ -2 \frac{\cos\theta}{\sin{\theta}}\ &\partial\theta\ \partial\phi \\
\end{array}
    \quad \Leftrightarrow \quad
\left[
\begin{array}{rclc}
    \ddot{\theta} &-& \sin\theta\cos\theta\ & \ddot{\phi} \\
    \ddot{\phi}   &+& 2 \cot\theta\ & \dot{\theta}\dot{\phi} \\
\end{array} 
\right] 
= \begin{bmatrix} 0 \\ 0 \end{bmatrix}
$$


for which we may use back-substitution to simplify the system. 

$$
\begin{split}
    (2) \quad \ddot{\phi}\ =\ -\frac{1}{\sin\theta\cos\theta}\ \ddot{\theta}\ 
    &\Rightarrow^* \quad \cot\theta\ \dot{\theta}\dot{\phi}\ =\ -\frac{1}{2\sin\theta\cos\theta}\ \ddot{\theta} \\
    &\Rightarrow^* \quad \dot{\theta}\dot{\phi}\ =\ -\frac{1}{2}\sec^2\theta\ \ddot{\theta} \\
    &\Rightarrow^* \quad \ddot{\theta} =\ -2\cos^2\theta\ \dot{\theta}\dot{\phi}\ \\
\end{split}
$$

Which may then be represented as:

$$
\begin{array}{rcrc}
    \ddot{\phi} &=& 2 \cot{\theta}\ &\dot\theta\dot\phi \\
    \ddot{\theta} &=& -2 \cos^2\theta\ &\dot\theta\dot\phi
\end{array}
    \quad \Leftrightarrow \quad 
\begin{array}{rcrl}
    \dot\theta\dot\phi &=& \frac{1}{2} \tan\theta\ &\ddot\phi \\
    \dot\theta\dot\phi &=& -\frac{1}{2} \sec^2\theta\ &\ddot\theta
\end{array}
$$

This can be simplified to a separable linear first-order system in two variables; 

$$
\left[
    \begin{array}{rcrc}
        \dot\theta &=& \tan\theta\ &\dot\phi \\
        \dot\phi &=& -\sec^2\theta &\dot\theta
    \end{array}
\right]
    \quad \Leftrightarrow \quad
\left[
    \begin{array}{rcrc}
        \dot\phi &=& \cot\theta\ &\dot\theta \\
        \dot\phi &=& -\sec^2\theta &\dot\theta
    \end{array}
\right]
$$

From here, we can either use another round of back-substitution, or implicitly
solve the separable equation. 

---

For funsies, we can also analytically solve the computer algebra form of the geodesic system of equations of motion, and express them as the Lagrangian of an Initial Value Problem in the coordinate system: 

In [32]:
sol = geodesic.solve_analytical()
sol


(ilt((g7377*theta_0 + Dtheta_0 + laplace(cos(y1(s))*sin(y1(s))*y4(s)^2, s, g7377))/g7377^2, g7377, s),
 ilt((g7377*phi_0 + Dphi_0 - 2*laplace(cos(y1(s))*y3(s)*y4(s)/sin(y1(s)), s, g7377))/g7377^2, g7377, s))

# [16]

Calculate the Reimann curvature tensor for a flat space in polar coordinates and on the sphere S2.

In [39]:
Γ = g.connection()

𝓡 = Γ.riemann()

𝓡.display_comp()


Riem(g)^theta_phi,theta,phi = sin(theta)^2 
Riem(g)^theta_phi,phi,theta = -sin(theta)^2 
Riem(g)^phi_theta,theta,phi = -1 
Riem(g)^phi_theta,phi,theta = 1

In [42]:
𝓡.display()


Riem(g) = sin(theta)^2 ∂/∂theta⊗dphi⊗dtheta⊗dphi - sin(theta)^2 ∂/∂theta⊗dphi⊗dphi⊗dtheta - ∂/∂phi⊗dtheta⊗dtheta⊗dphi + ∂/∂phi⊗dtheta⊗dphi⊗dtheta

__!! Note :> Key takeaway here is that both may be considered to locally be "flat", since the terms will cancel.__

# [17]

Compute the change in a vector due to transport around closed curve in S2

# [19]

You watch a muon speed to earth with energy 100 GeV. How long will it take to decay?  To reach the ground from the upper atmosphere?

# [20]

show that the momentum 4-vector has an invariant length of -m^2c^4

# [22]

Get the Ricci Tensor/scalar to show the field equation for huv is correct

# [23]

Get hmn for sun and show that it satisfies the linearized equation, get geodesic equation for light from hmn

# [24] 

Reproduce figure 4.5 from pg 140 of the text on the deflection of light.

# [25]

verify the solution of the scalar wave equation, the "plane wave" solution.

# [26]

Do Exercises 3, 4, and 5 in Section 3.1 of Ohanian.


## §3.1 ex. 3

>   Show that the gauge transformation (3.19) produces no change in $F_{\mu\nu}$.
>
>   $$ A^\nu \rightarrow A^\nu + \partial^\nu \Lambda \tag{3.19} $$
>
>   given: 
>
>   *   $$ F^{\mu\nu} \equiv \partial^\mu A^\nu - \partial^\nu A^\mu \tag{3.15} $$
>   *   $\Lambda$ is any arbitrary scalar function, the _gauge function_.  

## §3.1 ex. 4

>   Show that for a wave propagating in the $z$-direction, Eqs. (3.24) and (3.20), 
>   with $j^\nu = 0$, have the following linearly independent solutions:
>
>   $$ A^\nu = \epsilon_{(n)}^\nu \cos\left(\omega z - \omega t\right) \tag{3.26}$$
>
>   where $\epsilon_{(n)}^{\nu} : n \in \{1, 2, 3\}$ is one of the following vectors:
>
>   * $\epsilon_{(1)}^\nu = (0, 1, 0, 0)$
>   * $\epsilon_{(2)}^\nu = (0, 0, 1, 0)$
>   * $\epsilon_{(3)}^\nu = (1, 0, 0, 1)$
>
>   (Note :> The subscript $n$ has been enclosed in parentheses to indicate that it is 
>    not a tensor index; it is only a label that distinguishes between the four&ndash;
>    vectors above.)
>
>   Show that any other solution of the type $\cos\left(\omega z - \omega t\right)$
>   can be written as a linear combination of the three solutions, with the 
>   _polarization vectors_ $\epsilon_{(n)}^\nu$ listed above. 

## §3.1 ex. 5

>   Find the $\mathbf{E}$ and $\mathbf{B}$ fields for each of the solutions 
>   $\epsilon_{(n)}^\nu$, and show that the $\epsilon_{(3)}^\nu$ solution 
>   gives $\mathbf{E} = \mathbf{B} = 0$.
>
>   Hence, only the solutions with the first two polarization vectors $\epsilon_{(1)}^\nu$
>   and $\epsilon_{(2)}^\nu$ have any physical significance. 
>
>   These two polarization vectors correspond to the familiar two independent, 
>   perpendicular directions of polarization of an electromagnetic wave.  Show that
>   the solution with the polarization vector $\epsilon_{(3)}$\nu$ can be
>   completely eliminated by a gauge transformation, with $\Lambda = (1/\omega)
>   \sin\left(\omega z - \omega t\right)$, which leaves to $\overline{A}^\nu = 0$. 

# [27]

Compute the effect of a grav. wave traveling in the z-dir on a circle of masses arranged in the x-y plane.